In [1]:
import random
import networkx as nx
from matplotlib import pyplot as plt
from collections import Counter
import scipy as sc
from scipy.linalg import expm

import math
from utils.plotTools import plot_qwak
import os
import ast
import numpy as np
import json
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

In [4]:

def probability_distribution(M_t, init_state):
    """
    Computes the probability distribution p(t) for a graph with Laplacian matrix L,
    starting from the initial state init_state, at time t.
    """
    # Initial probability vector
    p0 = np.zeros(len(M_t))
    p0[init_state] = 1
    
    # Compute the transition matrix M(t)
    # M_t = transition_matrix(L, t)
    
    # Compute the new probability distribution p(t)
    p_t = np.round(np.dot(M_t, p0),5)
    
    return p_t


In [4]:
def first_passage_time_small_steps(G, gamma, start, end, delta_t):
    """
    Simulates a continuous-time random walk on a graph G with transition rate gamma,
    from start node to end node, using small time steps delta_t.
    Returns the first-passage time.
    """
    # Obtain the Laplacian matrix of the graph
    L = nx.laplacian_matrix(G).toarray()
    
    # The generator matrix H
    H = -gamma * L
    
    # The transition matrix for a small time step delta_t
    M_delta_t = expm(H * delta_t)
    
    # Initialize the current state to the start node
    current_state = start
    # Probability vector starting at the start node
    p = np.zeros(G.number_of_nodes())
    p[start] = 1
    
    # Accumulate time until the process reaches the end node
    time_elapsed = 0.0
    while current_state != end:
        # Apply the transition matrix to find the next state probabilities
        p = M_delta_t.dot(p)
        
        # Normalize to handle numerical errors that might cause the probabilities to sum to slightly less than 1
        p /= p.sum()
        
        # Choose the next state randomly according to the probabilities
        # next_state = np.random.choice(G.number_of_nodes(), p=p)
        
        # Update the current state and time
        current_state = next_state
        time_elapsed += delta_t
        
        # Reset the probability vector for the next iteration
        p = np.zeros(G.number_of_nodes())
        p[current_state] = 1
        M_delta_t = expm(H * time_elapsed)
    
    return time_elapsed

n = 100
G = nx.cycle_graph(n)


# Set the transition rate gamma
gamma = 1.0

# Starting and ending vertices
start_vertex = 0
end_vertex = 4

# Perform the walk and get the first-passage time
# time_taken = first_passage_time(G, gamma, start_vertex, end_vertex)
# print(f"Time taken to reach node {end_vertex} from node {start_vertex}: {time_taken}")

In [ ]:
def first_passage_time(G, gamma, start, end, delta_t):
    """
    Simulates a continuous-time random walk on a graph G from start node to end node.
    It uses small time steps delta_t and transition probabilities at each step to move to neighboring nodes.
    Returns the time taken to first reach the end node.
    """
    # Laplacian matrix of the graph
    L = nx.laplacian_matrix(G).toarray()
    
    # The generator matrix H
    H = -gamma * L
    
    # Initialize variables
    current_state = start
    time_elapsed = 0.0
    
    # Run the walk until the end state is reached
    while current_state != end:
        # Calculate the transition matrix for a small time step delta_t
        M_delta_t = expm(H * delta_t)
        
        # Extract the neighbors of the current state
        neighbors = list(G.neighbors(current_state))

        # Get the transition probabilities for the neighbors
        transition_probabilities = M_delta_t[current_state, neighbors]

        # Normalize probabilities to ensure they sum to 1
        if transition_probabilities.sum() > 0:
            transition_probabilities /= transition_probabilities.sum()
        else:
            # If there are no available transitions, raise an error
            raise ValueError(f"No available transitions from state {current_state}.")

        # Choose the next state from the neighbors based on the probabilities
        next_state = np.random.choice(neighbors, p=transition_probabilities)
        
        # Update the current state and the time elapsed
        current_state = next_state
        time_elapsed += delta_t

    return time_elapsed


n = 10
G = nx.cycle_graph(n)


# Set the transition rate gamma
gamma = 1.0

# Starting and ending vertices
start_vertex = 0
end_vertex = n//2

delta_t = 0.1

# Perform the walk and get the first-passage time
time_taken = first_passage_time(G, gamma, start_vertex, end_vertex, delta_t)
print(f"Time taken to reach node {end_vertex} from node {start_vertex}: {time_taken}")

In [ ]:
def transition_matrix(L, t):
    """
    Computes the transition matrix M(t) for a graph with Laplacian matrix L at time t
    using diagonal decomposition.
    """
    # Compute the eigenvalues and eigenvectors of the Laplacian matrix
    eigenvalues, eigenvectors = np.linalg.eigh(L)
    
    # Compute the diagonal matrix of the exponentiated eigenvalues
    exp_diagonal = np.diag(np.exp(-t * eigenvalues))
    
    # Compute the transition matrix using the eigenvectors and the exponentiated eigenvalues
    M_t = np.dot(eigenvectors, np.dot(exp_diagonal, eigenvectors.T))
    
    return M_t

def probability_distribution(M_t, init_state):
    """
    Computes the probability distribution p(t) for a graph with Laplacian matrix L,
    starting from the initial state init_state, at time t.
    """
    # Initial probability vector
    p0 = np.zeros(len(M_t))
    p0[init_state] = 1
    
    # Compute the transition matrix M(t)
    # M_t = transition_matrix(L, t)
    
    # Compute the new probability distribution p(t)
    p_t = np.round(np.dot(M_t, p0),5)
    
    return p_t

def choose_next_state(neighbors, probabilities):
    """
    Chooses the next state based on the given probabilities, which do not necessarily sum to 1.
    """
    cumulative_probabilities = np.cumsum(probabilities)
    print(cumulative_probabilities)
    print()
    random_number = np.random.rand() * cumulative_probabilities[-1]
    for neighbor, cumulative_probability in zip(neighbors, cumulative_probabilities):
        if random_number <= cumulative_probability:
            return neighbor
    return neighbors[-1]  # Fallback to the last neighbor in case of any numerical issues


def first_passage_time(G, gamma, start, end, delta_t):
    """
    Simulates a continuous-time random walk on a graph G from start node to end node.
    It uses small time steps delta_t and probability distributions at each step to move to neighboring nodes.
    Returns the time taken to first reach the end node.
    """
    # Laplacian matrix of the graph
    L = nx.laplacian_matrix(G).toarray() * gamma
    eigenvalues, eigenvectors = np.linalg.eigh(L)

    # Initialize variables
    current_state = start
    time_elapsed = delta_t
    
    # Run the walk until the end state is reached
    while current_state != end:
        # Calculate the probability distribution for a small time step delta_t
        exp_diagonal = np.diag(np.exp(-time_elapsed * eigenvalues))
        M_t = np.dot(eigenvectors, np.dot(exp_diagonal, eigenvectors.T))
        
        p_t = probability_distribution(M_t, current_state)
        
        # Extract the neighbors of the current state
        neighbors = list(G.neighbors(current_state))

        # Get the transition probabilities for the neighbors
        transition_probabilities = p_t[neighbors]

        # if transition_probabilities.sum() > 0:
        #     transition_probabilities /= transition_probabilities.sum()
        # else:
        #     # If there are no available transitions, raise an error
        #     raise ValueError(f"No available transitions from state {current_state}.")

        # Choose the next state from the neighbors based on the probabilities
        # print(neighbors)
        print(transition_probabilities)
        # print(p_t)
        # next_state = np.random.choice(neighbors, p=transition_probabilities)
        next_state = choose_next_state(neighbors, transition_probabilities)
        
        # Update the current state and the time elapsed
        current_state = next_state
        time_elapsed += delta_t

    return time_elapsed

n = 10
G = nx.cycle_graph(n)


# Set the transition rate gamma
gamma = 1.0

# Starting and ending vertices
start_vertex = 0
end_vertex = n//2

delta_t = 0.1

# Perform the walk and get the first-passage time
time_taken = first_passage_time(G, gamma, start_vertex, end_vertex, delta_t)
print(f"Time taken to reach node {end_vertex} from node {start_vertex}: {time_taken}")

In [18]:
def probability_distribution(M_t, init_state):
    """
    Computes the probability distribution p(t) for a graph with Laplacian matrix L,
    starting from the initial state init_state, at time t.
    """
    # Initial probability vector
    p0 = np.zeros(len(M_t))
    p0[init_state] = 1
    
    # Compute the transition matrix M(t)
    # M_t = transition_matrix(L, t)
    
    # Compute the new probability distribution p(t)
    p_t = np.round(np.dot(M_t, p0),5)
    
    return p_t


def efficient_transition_matrix(eigenvalues, eigenvectors, t):
    D_t = np.diag(np.exp(eigenvalues * t))
    return eigenvectors @ D_t @ np.linalg.inv(eigenvectors)

def first_passage_time(G, gamma, start, end, delta_t):
    """
    Simulates a continuous-time random walk on a graph G from start node to end node.
    It uses small time steps delta_t and probability distributions at each step to move to neighboring nodes.
    Returns the time taken to first reach the end node.
    """
    # Laplacian matrix of the graph
    # L = nx.laplacian_matrix(G).toarray() * gamma
    D = np.diag(list(dict(G.degree()).values()))
    A = nx.to_numpy_array(G)
    L = -gamma * (D - A)
    eigenvalues, eigenvectors = sc.linalg.eigh(L)

    # Initialize variables
    current_state = start
    time_elapsed = delta_t
    
    # Run the walk until the end state is reached
    while current_state != end:
        # Calculate the probability distribution for a small time step delta_t
        M_t = efficient_transition_matrix(eigenvalues,eigenvectors,time_elapsed)
        # print(M_t.round(2))
        p_t = probability_distribution(M_t, current_state)
        
        # Extract the neighbors of the current state
        neighbors = list(G.neighbors(current_state)) + [current_state]

        # Get the transition probabilities for the neighbors
        transition_probabilities = p_t[neighbors]
        # print(transition_probabilities)
        
        # Normalize transition probabilities
        if np.sum(transition_probabilities) > 0:
            transition_probabilities /= np.sum(transition_probabilities)
        else:
            # Handle the special case where the sum is 0
            # For example, stay in the current state or choose randomly
            transition_probabilities = np.ones(len(neighbors)) / len(neighbors)
        # print(transition_probabilities)
        # print()
        # print(p_t)
        next_state = np.random.choice(neighbors, p=transition_probabilities)
        
        # Update the current state and the time elapsed
        current_state = next_state
        time_elapsed += delta_t

    return time_elapsed

n = 10
G = nx.cycle_graph(n)


# Set the transition rate gamma
gamma = 1.0

# Starting and ending vertices
start_vertex = 0
end_vertex = n//2

delta_t = 0.1

# Perform the walk and get the first-passage time
time_taken = first_passage_time(G, gamma, start_vertex, end_vertex, delta_t)
print(f"Time taken to reach node {end_vertex} from node {start_vertex}: {time_taken}")

Time taken to reach node 5 from node 0: 1.8000000000000005


In [19]:
import time

def average_hitting_time_for_range(n_range, samples, gamma, delta_t):
    """
    Calculates the average hitting time for graphs of sizes within n_range,
    performing a specified number of samples for each graph size.

    :param n_range: Range of graph sizes to test (e.g., range(5, 10))
    :param samples: Number of samples to perform for each graph size
    :param gamma: Transition rate parameter for the random walk
    :param delta_t: Time step for the random walk
    :return: List with average hitting times for each graph size
    """
    avg_hitting_times = []

    for n in n_range:
        # print(n)
        total_time = 0
        end_vertex = n // 2

        for _ in range(samples):
            G = nx.cycle_graph(n)
            start_vertex = 0

            time_taken = first_passage_time(G, gamma, start_vertex, end_vertex, delta_t)
            total_time += time_taken

        avg_hitting_times.append(total_time / samples)

    return avg_hitting_times

n_range = range(5, 15)  # Range of graph sizes
samples = 100           # Number of samples for each size
gamma = 1.0             # Transition rate
delta_t = 0.1           # Time step

# Start timing
start_time = time.time()

# Call the function
average_times = average_hitting_time_for_range(n_range, samples, gamma, delta_t)

# End timing
end_time = time.time()

# Calculate and print the elapsed time
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time} seconds")


Elapsed time: 3.1290273666381836 seconds


In [24]:
import cupy as cp
import networkx as nx

def probability_distribution(M_t, init_state):
    p0 = cp.zeros(len(M_t))
    p0[init_state] = 1
    p_t = cp.round(cp.dot(M_t, p0), 5)
    return p_t

def efficient_transition_matrix(eigenvalues, eigenvectors, t):
    D_t = cp.diag(cp.exp(eigenvalues * t))
    return cp.matmul(cp.matmul(eigenvectors, D_t), cp.linalg.inv(eigenvectors))

def first_passage_time2(G, gamma, start, end, delta_t):
    D = cp.diag(cp.array(list(dict(G.degree()).values())))
    A = cp.array(nx.to_numpy_array(G))
    L = -gamma * (D - A)

    # Compute eigenvalues and eigenvectors using CuPy
    eigenvalues, eigenvectors = np.linalg.eigh(L)
    # print(type(eigenvalues))

    current_state = start
    time_elapsed = delta_t
    
    while current_state != end:
        M_t = efficient_transition_matrix(eigenvalues, eigenvectors, time_elapsed)
        p_t = probability_distribution(M_t, current_state)
        
        neighbors = list(G.neighbors(current_state)) + [current_state]
        transition_probabilities = p_t[neighbors].get()  # Convert to NumPy array for random choice

        if transition_probabilities.sum() > 0:
            transition_probabilities /= transition_probabilities.sum()
        else:
            transition_probabilities = cp.ones(len(neighbors)) / len(neighbors)

        next_state = np.random.choice(neighbors, p=transition_probabilities)
        current_state = next_state.item()  # Convert CuPy scalar to Python scalar
        time_elapsed += delta_t

    return time_elapsed


In [25]:
import time

def average_hitting_time_for_range2(n_range, samples, gamma, delta_t):
    """
    Calculates the average hitting time for graphs of sizes within n_range,
    performing a specified number of samples for each graph size.

    :param n_range: Range of graph sizes to test (e.g., range(5, 10))
    :param samples: Number of samples to perform for each graph size
    :param gamma: Transition rate parameter for the random walk
    :param delta_t: Time step for the random walk
    :return: List with average hitting times for each graph size
    """
    avg_hitting_times = []

    for n in n_range:
        # print(n)
        total_time = 0
        end_vertex = n // 2

        for _ in range(samples):
            G = nx.cycle_graph(n)
            start_vertex = 0

            time_taken = first_passage_time2(G, gamma, start_vertex, end_vertex, delta_t)
            total_time += time_taken

        avg_hitting_times.append(total_time / samples)

    return avg_hitting_times

n_range = range(5, 15)  # Range of graph sizes
samples = 100           # Number of samples for each size
gamma = 1.0             # Transition rate
delta_t = 0.1           # Time step

# Start timing
start_time = time.time()

# Call the function
average_times = average_hitting_time_for_range2(n_range, samples, gamma, delta_t)

# End timing
end_time = time.time()

# Calculate and print the elapsed time
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time} seconds")


Elapsed time: 47.01345443725586 seconds


In [28]:
import cupy as cp
import networkx as nx

def probability_distribution(M_t, init_state):
    p0 = cp.zeros(len(M_t))
    p0[init_state] = 1
    p_t = cp.round(cp.dot(M_t, p0), 5)
    return p_t

def efficient_transition_matrix(eigenvalues, eigenvectors, t):
    D_t = cp.diag(cp.exp(eigenvalues * t))
    return cp.matmul(cp.matmul(eigenvectors, D_t), cp.linalg.inv(eigenvectors))

def gpu_random_choice(probs):
    cumulative = cp.cumsum(probs)
    rand = cp.random.rand()
    return cp.where(cumulative >= rand)[0][0]

def first_passage_time2(G, gamma, start, end, delta_t):
    D = cp.diag(cp.array(list(dict(G.degree()).values())))
    A = cp.array(nx.to_numpy_array(G))
    L = -gamma * (D - A)

    # Compute eigenvalues and eigenvectors using CuPy
    eigenvalues, eigenvectors = cp.linalg.eigh(L)

    current_state = start
    time_elapsed = delta_t
    
    while current_state != end:
        M_t = efficient_transition_matrix(eigenvalues, eigenvectors, time_elapsed)
        p_t = probability_distribution(M_t, current_state)
        
        neighbors = cp.array(list(G.neighbors(current_state)) + [current_state])
        transition_probabilities = p_t[neighbors]

        if transition_probabilities.sum() > 0:
            transition_probabilities /= transition_probabilities.sum()
        else:
            transition_probabilities = cp.ones(len(neighbors)) / len(neighbors)

        next_state = np.random.choice(neighbors, p=transition_probabilities)
        current_state = next_state.item()  # Convert CuPy scalar to Python scalar
        time_elapsed += delta_t

    return time_elapsed


In [29]:
import time

def average_hitting_time_for_range2(n_range, samples, gamma, delta_t):
    """
    Calculates the average hitting time for graphs of sizes within n_range,
    performing a specified number of samples for each graph size.

    :param n_range: Range of graph sizes to test (e.g., range(5, 10))
    :param samples: Number of samples to perform for each graph size
    :param gamma: Transition rate parameter for the random walk
    :param delta_t: Time step for the random walk
    :return: List with average hitting times for each graph size
    """
    avg_hitting_times = []

    for n in n_range:
        # print(n)
        total_time = 0
        end_vertex = n // 2

        for _ in range(samples):
            G = nx.cycle_graph(n)
            start_vertex = 0

            time_taken = first_passage_time2(G, gamma, start_vertex, end_vertex, delta_t)
            total_time += time_taken

        avg_hitting_times.append(total_time / samples)

    return avg_hitting_times

n_range = range(5, 15)  # Range of graph sizes
samples = 100           # Number of samples for each size
gamma = 1.0             # Transition rate
delta_t = 0.1           # Time step

# Start timing
start_time = time.time()

# Call the function
average_times = average_hitting_time_for_range2(n_range, samples, gamma, delta_t)

# End timing
end_time = time.time()

# Calculate and print the elapsed time
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time} seconds")


TypeError: Implicit conversion to a NumPy array is not allowed. Please use `.get()` to construct a NumPy array explicitly.